In [1]:
!pip install unidecode tensorboardX
!git clone --recursive https://github.com/NVIDIA/flowtron

     |████████████████████████████████| 245kB 6.6MB/s 
     |████████████████████████████████| 317kB 15.6MB/s 
Cloning into 'flowtron'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 171 (delta 14), reused 16 (delta 5), pack-reused 135
Receiving objects: 100% (171/171), 2.81 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Submodule 'apex' (https://github.com/NVIDIA/apex) registered for path 'apex'
Submodule 'tacotron2' (https://github.com/NVIDIA/tacotron2) registered for path 'tacotron2'
Cloning into '/content/flowtron/apex'...
remote: Enumerating objects: 30, done.        
remote: Counting objects: 100% (30/30), done.        
remote: Compressing objects: 100% (30/30), done.        
remote: Total 7865 (delta 7), reused 10 (delta 0), pack-reused 7835        
Receiving objects: 100% (7865/7865), 13.99 MiB | 23.11 MiB/s, done.
Resolving deltas: 100% (5354/5354), done.
Clon

In [2]:
%cd flowtron

/content/flowtron


In [3]:
import sys
import json
import torch
import matplotlib

from data import Data
import IPython.display as ipd
from flowtron import Flowtron
import matplotlib.pylab as plt
from train import update_params
from torch.distributions import Normal

sys.path.insert(0, "tacotron2")
sys.path.insert(0, "tacotron2/waveglow")

from denoiser import Denoiser